In [1]:
import import_ipynb
from library import *

importing Jupyter notebook from library.ipynb
importing Jupyter notebook from job.ipynb
importing Jupyter notebook from jobqueue.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from dataplane.ipynb


In [2]:
class node:
    def __init__(self, capacity):
        self.capacity = capacity
        self.waiting_queue = collections.deque()
        self.cur_proc_job = -1
        self.cur_proc_subtask = -1
    
    def push(self, job):
        self.waiting_queue.append(job)
        
    def pop(self):
        return self.waiting_queue.popleft()

    def get_info(self, index):
        if len(self.waiting_queue) == 0:
            return
        print(index)
        for job in self.waiting_queue:
            print(job.subtasks[0], end=' ')
        print()

In [3]:
class link:
    def __init__(self, capacity):
        self.capacity = capacity
        self.waiting_queue = collections.deque()
        self.cur_remain = 0
        self.waiting = 0
        
    def push(self, job):
        self.waiting_queue.append(job)
        
    def pop(self):
        return self.waiting_queue.popleft()
    
    def get_waiting(self):
        return self.waiting
    
    def get_info(self, index):
        if len(self.waiting_queue) == 0:
            return
        else:
            print(index)
            for job in self.waiting_queue:
                print(job.subtasks[0], end=' ')
            print()

In [3]:
class network:
    def __init__(self):
        self.nodes = []
        self.links = {}
        self.adjacent = [[1, 2], [0, 2, 3], [0, 1, 3, 4], [1, 2, 4], [2, 3]]
        for n in range(5):
            self.nodes.append(node(n * 10 + 10))
        for src in range(5):
            for dst in self.adjacent[src]:
                self.links[f'{src}{dst}'] = link(10)
                
                
    def step(self, time):
        #print("--------------------------------------------------------------")
        completed_job_index = []
        job_index1 = self.step_link(time)
        job_index2 = self.step_node(time)
        completed_job_index.extend(job_index1)
        completed_job_index.extend(job_index2)
        return completed_job_index
        '''
        print("node)")
        for index, node in enumerate(self.nodes):
            node.get_info(index)
        print("\nlink)")
        for key, link in self.links.items():
            link.get_info(key)
        print('\n')
        '''
            

    
    def step_node(self, time):
        completed_job_index = []
        for node_index, node in enumerate(self.nodes):
            if len(node.waiting_queue) == 0:
                continue
            job = node.waiting_queue[0]
            rem_tasks = job.subtasks
            rem_tasks[0].comp_demand = max(0, rem_tasks[0].comp_demand-(node.capacity*0.1))
            job.progress += 0.1*node.capacity
            job.comp_delta += 0.1*node.capacity
            
            # record current processing task
            node.cur_proc_job = job.index
            node.cur_proc_subtask = rem_tasks[0].order
            
            if rem_tasks[0].comp_demand == 0:
                node.cur_proc_job = -1
                node.cur_proc_task = -1
                if len(job.offloading) == 1:
                    #print(f'current time : {time}, job birth : {job.birth}')
                    #print(f'completion time {(time-job.birth)/10}')
                    completed_job_index.append(job.index)
                    node.pop()
                else:
                    start, end = job.offloading[0], job.offloading[1]
                    if start != end:
                        path = self.routing(start, end)
                        job.routing = path
                        self.links[f'{path[0]}{path[1]}'].push(job)
                        self.links[f'{path[0]}{path[1]}'].waiting += rem_tasks[0].link_demand
                        #print(f"node to flow : {path[0]} {path[1]}")
                        del job.routing[0]
                        node.pop()
                    else:
                        del rem_tasks[0]
                    del job.offloading[0]
                
                    
        return completed_job_index
                    
                    
                    
    def step_link(self, time):
        completed_job_index = []
        for key, link in self.links.items():
            if len(link.waiting_queue) == 0:
                continue
            job = link.waiting_queue[0]
            rem_tasks = job.subtasks
            rem_tasks[0].link_demand = max(0, rem_tasks[0].link_demand-(link.capacity*0.1))
            link.waiting = max(0, link.waiting-(link.capacity*0.1))
            
            if rem_tasks[0].link_demand == 0:
                # 해당 layer의 destination node에 도달했다면
                if len(job.routing) == 1:
                    del rem_tasks[0]
                    if len(rem_tasks) != 0:
                        node = job.routing[0]
                        self.nodes[node].push(job)
                    # job completion
                    else:
                        #print(f'current time : {time}, job birth : {job.birth}')
                        #print(f'completion time {(time-job.birth)/10}')
                        completed_job_index.append(job.index)
                else:
                    rem_tasks[0].link_demand = rem_tasks[0].ori_link_demand # recover data size
                    start, end = job.routing[0], job.routing[1]
                    #print(f"link to flow : {start} {end}")
                    self.links[f'{start}{end}'].push(job)
                    self.links[f'{start}{end}'].waiting += rem_tasks[0].link_demand
                    del job.routing[0]
                    
                link.pop()
        
        return completed_job_index
                
         
        
    def routing(self, start, end):
        graph = [[] for _ in range(5)]
        pre = [0]*5
        distance = [10000]*5
        for src in range(5):
            for dst in self.adjacent[src]:
                cost = self.links[f'{src}{dst}'].get_waiting()/self.links[f'{src}{dst}'].capacity
                graph[src].append((dst, cost))
                
        q = [(0, start)]
        distance[start] = 0
        while q:
            dist, idx = heapq.heappop(q)
            if dist != distance[idx]:
                continue
            for nidx, cost in graph[idx]:
                if distance[nidx] > dist+cost:
                    distance[nidx] = dist+cost
                    pre[nidx] = idx
                    heapq.heappush(q, (distance[nidx], nidx))
                    
        path = []
        cur_node = end
        while cur_node != start:
            path.append(cur_node)
            cur_node = pre[cur_node]
        path.append(start)
        path.reverse()
        return path